In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import torch
import mlflow
import dagshub
import numpy as np
import torchmetrics
import torch.nn as nn
import torch.optim as optim
import src.utils.data as data
import pytorch_lightning as pl
import torchvision.models as models

from tqdm.notebook import tqdm
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torchmetrics.classification import BinaryPrecision, BinaryRecall

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [4]:
run_id = "115ecd0ffec84cbfbfb6431b4d879464" # Change to model

mlflow.set_tracking_uri('https://dagshub.com/FlareSense/Flaresense.mlflow')
model = mlflow.pytorch.load_model(f"runs:/{run_id}/model/").to(device)
model.eval();

In [5]:
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=32,
    num_workers=0,
    val_ratio=0.2,
    test_ratio=0.2,
    split_by_date=True,
    filter_instruments=["australia_assa_02"],    
)
data_module.setup()

val_loader = data_module.val_dataloader()

In [6]:
val_labels_list = []
val_preds_list = []
with torch.no_grad():
    for batch in tqdm(val_loader):
        images, info = batch
        images = images.to(device)
        binary_labels = [0 if label == "no_burst" else 1 for label in info['label']]
        binary_labels = torch.tensor(binary_labels).int().view(-1, 1)
        binary_labels = binary_labels.to(device)
        
        outputs = model(images.expand(-1, 3, -1, -1))
        predictions = (outputs >= 0.5).int()
        
        val_labels_list.append(binary_labels)
        val_preds_list.append(predictions)

  0%|          | 0/127 [00:00<?, ?it/s]

In [7]:
len(val_preds_list)

127

In [8]:
len(val_loader)

127

In [9]:
# Alle Validierungsdaten wurden gesammelt
val_labels = torch.cat(val_labels_list, dim=0)
val_preds = torch.cat(val_preds_list, dim=0)

In [10]:
# Initialisieren Sie den ConfusionMatrix-Metriken von torchmetrics
confmat_metric = torchmetrics.ConfusionMatrix(num_classes=2, task="binary").to(device)
confmat_metric(val_preds, val_labels)

tensor([[3885,    6],
        [  49,   94]], device='cuda:0')